###### Import Dependencies & Chrome Driver

In [1]:
# import dependencies
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint
from splinter import Browser
import time

In [2]:
# choose path to driver (Chrome v91)
executable_path = {'executable_path' : 'chromedriver'}
browser = Browser(
    'chrome', 
    **executable_path, 
    headless = False
)

# delay to allow time for page to load
time.sleep(5)

###### Scraping Most Recent Article

In [3]:
# navigate browser to url
browser.visit('https://mars.nasa.gov/news/')

# delay to allow time for page to load
time.sleep(5)

# create beautifulsoup object
article_soup = BeautifulSoup(browser.html, 'html.parser')

In [4]:
# narrow HTML elements by 'article'
article_result = article_soup.find('article')

# retrieve most recent article date, title, paragraph
news_date = article_result.find('div', class_ = 'list_date').get_text()
news_title = article_result.find('div', class_ = 'content_title').get_text()
news_p = article_result.find('div', class_ = 'article_teaser_body').get_text()

###### Scraping Featured Image

In [5]:
# navigate browser to url, click full-res image
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')
browser.links.find_by_partial_text('FULL IMAGE').click()

# delay to allow time for page to load
time.sleep(5)

# create beautifulsoup object
image_soup = BeautifulSoup(browser.html, 'html.parser')

In [6]:
# extract image source url
image_result = image_soup.find(class_ = 'fancybox-image')['src']

In [7]:
# append urls
featured_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + image_result

###### Converting HTML table to DataFrame, convert back to HTML

In [8]:
# navigate browser to url
browser.visit('https://space-facts.com/mars/')

# delay to allow time for page to load
time.sleep(5)

# create beautifulsoup object
fact_soup = BeautifulSoup(browser.html, 'html.parser')

In [9]:
# convert html table to dataframe
fact_df = pd.read_html('https://space-facts.com/mars/')[0]

In [10]:
# transform dataframe for website
fact_df.columns = ['Description', 'Mars']
fact_df.set_index('Description', inplace = True)

In [11]:
# convert dataframe to html
fact_html = fact_df.to_html(index = False, header = False)

###### Mars Hemispheres

In [12]:
# navigate browser to url
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

# delay to allow time for page to load
time.sleep(5)

# create beautifulsoup object
soup = BeautifulSoup(browser.html, 'html.parser')

In [13]:
# grab hemisphere HTML elements
results = soup.find_all('div', class_ = 'item')

In [14]:
core_url = 'https://astrogeology.usgs.gov'
hemisphere_image_urls = []
# Loop through each hemisphere item
for item in results:
    # Error handling
    try:
        # Extract title
        hemi = item.find('div', class_ = 'description')
        title = hemi.h3.text
        # Extract image url
        hemi_url = hemi.a['href']
        browser.visit(core_url + hemi_url)
        soup = BeautifulSoup(browser.html, 'html.parser')
        image_src = soup.find('li').a['href']
        # Create dictionary for title and url
        hemi_dict = {'title' : title, 'image_url' : image_src}
        hemisphere_image_urls.append(hemi_dict)
    except Exception as e:
        print(e)

In [15]:
# close browser
browser.quit()